ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [25]:
pip install numpy==1.23.5 scikit-learn==1.4.2  joblib pandas


Note: you may need to restart the kernel to use updated packages.


In [3]:

import json

#GETTING MY ID:
id = open('../id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"]
#my_id

In [2]:
!python3 -m pip install pip==24.0



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\vhgar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!python --version

Python 3.11.9


In [4]:
!pip install azureml-sdk

In [4]:
## STEP 1: Set up the workspace.
from azureml.core import Workspace

ws = Workspace.create(name="workspace_final_final",
                      subscription_id = my_id,
                      resource_group = "resource_group_bank_1_2_3_4",
                      location = "centralindia")



WorkspaceException: WorkspaceException:
	Message: Workspace with name 'workspace_final_final' already exists under resource group with name 'resource_group_bank_1_2_3_4'.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Workspace with name 'workspace_final_final' already exists under resource group with name 'resource_group_bank_1_2_3_4'."
    }
}

In [ ]:
from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="../best_model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [ ]:
from azureml.core.model import Model

mname_scaler = "model_scaler"
registered_scaler = Model.register(model_path="../scaler.pkl",
                                  model_name=mname_scaler,
                                  workspace=ws)

Registering model model_scaler


In [ ]:
scorepy = """
import json 
import joblib
import pandas as pd
import numpy as np
import traceback
import sklearn
from azureml.core.model import Model

def embbed(d):
    try:
        d.columns = d.columns.str.strip()  # Limpiar nombres de columnas
        required_columns = [
            'Net Income to Total Assets', 'ROA(A) before interest and % after tax',
            'ROA(B) before interest and depreciation after tax',
            'ROA(C) before interest and depreciation before interest',
            'Net worth/Assets', 'Debt ratio %',
            'Persistent EPS in the Last Four Seasons',
            'Retained Earnings to Total Assets',
            'Net profit before tax/Paid-in capital',
            'Per Share Net profit before tax (Yuan �)',
            'Current Liability to Assets', 'Working Capital to Total Assets',
            "Net Income to Stockholder's Equity", 'Borrowing dependency',
            'Current Liability to Current Assets', 'Liability to Equity',
            'Net Value Per Share (A)', 'Net Value Per Share (B)',
            'Net Value Per Share (C)', 'Current Liability to Equity'
        ]

        missing_columns = [col for col in required_columns if col not in d.columns]
        if missing_columns:
            raise KeyError(f"Missing columns: {missing_columns}")

        return d[required_columns]

    except Exception as e:
        print(f"[ERROR] Unexpected error in embbed(): {e}")
        traceback.print_exc()
        return None

def init():
    global model, scaler

    try:
        print("[INFO] Loading model and scaler...")
        print(f"[INFO] NumPy version: {np.__version__}")
        print(f"[INFO] Scikit-learn version: {sklearn.__version__}")

        # Reemplaza 'model_name_here' por el nombre real de tu modelo registrado en Azure ML.
        model_path = Model.get_model_path('model')
        scaler_path = Model.get_model_path('model_scaler')

        print(f"[INFO] Loading model from {model_path}")
        model = joblib.load(model_path)
        print(f"[INFO] Loaded model from {model_path}!!")

        print(f"[INFO] Loading scaler from {scaler_path}")
        scaler = joblib.load(scaler_path)
        print(f"[INFO] Loaded scaler from {scaler_path}!!")

        print("[INFO] Model and scaler loaded successfully.")

    except Exception as e:
        print(f"[ERROR] Failed to load model or scaler: {e}")
        traceback.print_exc()
        model, scaler = None, None  # Aseguramos que sean None si falla la carga

def run(raw_data):
    global model, scaler

    try:
        if model is None or scaler is None:
            raise RuntimeError("[ERROR] Model or scaler not initialized. Check logs for errors.")

        data_dict = json.loads(raw_data)
        if "data" not in data_dict:
            raise ValueError("[ERROR] Input JSON must contain a 'data' key.")

        data = data_dict["data"]
        if not isinstance(data, list):
            raise ValueError("[ERROR] Expected 'data' to be a list of feature dictionaries.")

        df = pd.DataFrame(data)

        print("[INFO] Received data shape:", df.shape)
        print("[INFO] Data columns:", df.columns.tolist())

        embedded_data = embbed(df)
        if embedded_data is None:
            raise ValueError("[ERROR] Data embedding failed due to missing columns.")

        # Verificar si hay valores NaN antes de transformar
        if embedded_data.isnull().values.any():
            raise ValueError("[ERROR] Input data contains NaN values. Please clean your input.")

        scaled_data = scaler.transform(embedded_data)
        print("[INFO] Scaled data shape:", scaled_data.shape)

        result_finals = model.predict(scaled_data)
        print("[INFO] Model prediction output:", result_finals)

        return json.dumps({"predictions": result_finals.tolist()})

    except json.JSONDecodeError as e:
        error_msg = f"[ERROR] Invalid JSON input: {e}"
    except ValueError as e:
        error_msg = str(e)
    except Exception as e:
        error_msg = f"[ERROR] Unexpected error in run(): {e} {traceback.format_exc()}"

    print(error_msg)
    return json.dumps({"error": error_msg})

"""

In [ ]:


# Guardar el script en un archivo
file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()


UnicodeEncodeError: 'charmap' codec can't encode character '\ufffd' in position 759: character maps to <undefined>

In [ ]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-for-bank-1-3")


from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['numpy==1.23.5','scikit-learn=1.4.2', 'joblib', 'pandas']
)


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model, registered_scaler],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\vhgar\AppData\Local\Temp\ipykernel_52412\3279380664.py:21: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [ ]:
virtual_env

{
    "assetId": null,
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": "2g"
    },
    "environmentVariables": {
        "AZUREML_ENTRY_SCRIPT": "score.py",
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "env-for-bank-1-3",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": 

In [ ]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 761ad22e-fb18-4ead-9734-bff0b52820e6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image dfee6c5181ec4630b17cb20447e40f65.azurecr.io/azureml/azureml_bf404520237a3ff2b749b9ec64b63678 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 761ad22e-fb18-4ead-9734-bff0b52820e6
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: service. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image dfee6c5181ec4630b17cb20447e40f65.azurecr.io/azureml/azureml_bf404520237a3ff2b749b9ec64b63678 locally. Please refer to

In [ ]:
models = Model.list(workspace=ws)
for model in models:
    print(model.name, model.version)


model_scaler 1
model 1


In [ ]:
print(service.get_logs())


/bin/bash: /azureml-envs/azureml_b64eb7c138348198653a4668ab6e818b/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_b64eb7c138348198653a4668ab6e818b/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_b64eb7c138348198653a4668ab6e818b/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-04T05:43:12,040116552+00:00 - rsyslog/run 
bash: /azureml-envs/azureml_b64eb7c138348198653a4668ab6e818b/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-04T05:43:12,046967030+00:00 - gunicorn/run 
2025-04-04T05:43:12,048353396+00:00 | gunicorn/run | 
2025-04-04T05:43:12,050061784+00:00 | gunicorn/run | ###############################################
2025-04-04T05:43:12,052341056+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-04T05:43:12,054129215+00:00 - nginx/run 
2025-04-04T05:43:12,055472301+00:00 | gunico

In [ ]:
service.delete()


NameError: name 'service' is not defined

In [ ]:
pip install numpy==1.26.4 scikit-learn==1.4.2  joblib pandas


  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-dataset-runtime 1.59.0 requires numpy!=1.19.4,<1.24; sys_platform == "win32", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import joblib
import sklearn

print(f"Usando scikit-learn versión: {sklearn.__version__}")

try:
    model = joblib.load("../scaler.pkl")
    print("✅ Modelo cargado exitosamente")
except Exception as e:
    print(f"❌ Error al cargar el modelo: {e}")


Usando scikit-learn versión: 1.4.2
✅ Modelo cargado exitosamente


In [ ]:
pip uninstall scikit-learn


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import joblib
import sklearn
import traceback
import numpy
print(f"Usando scikit-learn versión: {sklearn.__version__}")

print(f"Usando numpy versión: {numpy.__version__}")

try:
    # Intenta cargar el modelo
    model = joblib.load("../best_model.pkl")
    print("✅ Modelo cargado exitosamente")
except Exception as e:
    # Si ocurre un error, muestra el mensaje de error y la traza completa
    print(f"❌ Error al cargar el modelo: {e}")
    traceback.print_exc()  # Esto imprimirá la traza completa del error


Usando scikit-learn versión: 1.4.2
Usando numpy versión: 1.26.4
✅ Modelo cargado exitosamente
